# Let's make embeddings from the descriptions we downloaded in Download311fromAPI.ipynb

In [6]:
!pip install -q keras-core 
!pip install -q --upgrade keras-nlp

^C
ERROR: Operation cancelled by user


In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

import keras_nlp
import keras_core as keras
import pandas as pd
import numpy as np

In [4]:

# Initialize tokenizer and preprocessor
tokenizer = keras_nlp.models.BertTokenizer.from_preset("bert_tiny_en_uncased")
preprocessor = keras_nlp.models.BertPreprocessor(tokenizer=tokenizer)

# Initialize the BERT backbone model from a preset
bert_backbone = keras_nlp.models.BertBackbone.from_preset("bert_tiny_en_uncased")

# Initialize an empty DataFrame to hold the final embeddings
final_embeddings_df = pd.DataFrame()

# Load the data
boston_311_data = pd.read_csv('./boston_311_data.csv')

# Loop through the data in batches
batch_size = 10000  # Adjust this based on your machine's capabilities
for i in range(0, len(boston_311_data), batch_size):
    batch_data = boston_311_data.iloc[i:i+batch_size]
    preprocessed_data = preprocessor(batch_data['description'].astype(str))
    
    # Generate embeddings
    embeddings = bert_backbone.predict(preprocessed_data)
    
    # Extract [CLS] embeddings and pooled_output
    cls_embeddings = embeddings['sequence_output'][:, 0, :]
    pooled_output = embeddings['pooled_output']
    
    # Combine the embeddings and service_request_id into a single DataFrame
    embeddings_df = pd.DataFrame({
        'service_request_id': batch_data['service_request_id'].values,
        'cls_embedding': list(cls_embeddings),
        'pooled_embedding': list(pooled_output)
    })

    # print(embeddings_df.head(5))
    
    # Append to the final DataFrame
    final_embeddings_df = pd.concat([final_embeddings_df, embeddings_df], ignore_index=True)

# Optional: Save to CSV
final_embeddings_df.to_csv('./cls_and_pooled_embeddings_with_service_id.csv', index=False)


ModuleNotFoundError: No module named 'keras_nlp'